In [10]:
# Use the Azure Machine Learning data source package
from azureml.dataprep import datasource
from azureml.logging import get_azureml_logger

import pyspark
from pyspark.ml.tuning import *
from pyspark.sql.types import *

# Use the Azure Machine Learning data collector to log various metrics
logger = get_azureml_logger()

In [11]:
# This call will load the referenced data source and return a DataFrame.
# If run in a PySpark environment, this call returns a
# Spark DataFrame. If not, it returns a Pandas DataFrame.
df = datasource.load_datasource('ratings.dsource')

In [12]:
from pyspark.ml.recommendation import ALS
als = ALS() \
    .setUserCol("userId") \
    .setRatingCol("rating") \
    .setItemCol("movieId") \

alsModel = als.fit(df)

In [14]:
!rm -r ./outputs/model

In [15]:
alsModel.save("./outputs/model")

In [16]:
!ls outputs/model

itemFactors  metadata  userFactors


In [17]:
userRecs = alsModel.recommendForAllUsers(100)

In [46]:
# Need to create CosmosDB Collection before insertion
import pydocumentdb
from pydocumentdb import document_client
from pydocumentdb import documents
import datetime

masterKey = 'oX6tWPep8FCah8RM258s7cC3x9Kl8tWdbDxmNknXCP34ShW1Ag1ladvb5QWuBmMxuRISBO2HfrRFv3QeJYCSYg=='
host = 'https://dcibrecommendationhack.documents.azure.com:443/'
database = 'recommendation_engine'
collection = 'user_recommendations_100_part4'

database_link = 'dbs/' + database

client = document_client.DocumentClient(host, {'masterKey': masterKey})

In [47]:
coll = { 'id': collection, 'partitionKey': {'paths': ['/userId']} }
collection_options = { 'offerThroughput': 1000 }
client.CreateCollection(database_link, coll, collection_options )

{'_conflicts': 'conflicts/',
 '_docs': 'docs/',
 '_etag': '"00003800-0000-0000-0000-5a8324420000"',
 '_rid': 'iqYGAOWRjQM=',
 '_self': 'dbs/iqYGAA==/colls/iqYGAOWRjQM=/',
 '_sprocs': 'sprocs/',
 '_triggers': 'triggers/',
 '_ts': 1518543938,
 '_udfs': 'udfs/',
 'id': 'user_recommendations_100_part4',
 'indexingPolicy': {'automatic': True,
  'excludedPaths': [],
  'includedPaths': [{'indexes': [{'dataType': 'Number',
      'kind': 'Range',
      'precision': -1},
     {'dataType': 'String', 'kind': 'Hash', 'precision': 3}],
    'path': '/*'}],
  'indexingMode': 'consistent'},
 'partitionKey': {'kind': 'Hash', 'paths': ['/userId']}}

In [48]:

# Write configuration
writeConfig = {
"Endpoint" : host,
"Masterkey" : masterKey,
"Database" : database,
"Collection" : collection,
"Upsert" : "true"
}
userRecs.select("userId", "recommendations.movieId").write.format("com.microsoft.azure.cosmosdb.spark").options(**writeConfig).save()